In [ ]:
import cv2

img = cv2.imread("input.jpg")


In [ ]:
import pytesseract
from PIL import Image

# If tesseract is not in PATH, set it manually
# pytesseract.pytesseract.tesseract_cmd = r"/usr/bin/tesseract"

image_path = "/home/kawsar/Desktop/Image-Masking/POC-table-masking/src/images/ss-1.jpeg"
image_path = "/home/kawsar/Downloads/selected_sap_images/9325_screenshot.png"

img = Image.open(image_path)

text = pytesseract.image_to_string(img, lang="eng", config="--psm 12")
# remove empty lines
text = "\n".join([line for line in text.split("\n") if line.strip() != ""])

print(text)


In [ ]:
import cv2
import pytesseract

# image_path = "image.png"

img = cv2.imread(image_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Optional preprocessing
gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

text = pytesseract.image_to_string(gray, lang="eng", config="--psm 12")
text = "\n".join([line for line in text.split("\n") if line.strip() != ""])

print(text)


In [4]:
import cv2
import pytesseract
from pytesseract import Output

image_path = "/home/kawsar/Desktop/Image-Masking/POC-table-masking/outputs/cropped_images/cropped_5.png"
img = cv2.imread(image_path)

data = pytesseract.image_to_data(
    img,
    lang="eng",
    config="--oem 3 --psm 12",
    output_type=pytesseract.Output.DICT
)

lines = {}

# -----------------------------
# Group words into lines
# -----------------------------
for i in range(len(data["text"])):
    text = data["text"][i].strip()
    conf = int(data["conf"][i])

    # level 5 = word
    if not text or conf <= 0 or data["level"][i] != 5:
        continue

    key = (
        data["block_num"][i],
        data["par_num"][i],
        data["line_num"][i]
    )

    lines.setdefault(key, []).append(i)

line_texts = []

# -----------------------------
# Build line-level text + box
# -----------------------------
for indices in lines.values():
    texts = [data["text"][i] for i in indices]

    x_min = min(data["left"][i] for i in indices)
    y_min = min(data["top"][i] for i in indices)
    x_max = max(data["left"][i] + data["width"][i] for i in indices)
    y_max = max(data["top"][i] + data["height"][i] for i in indices)

    line_texts.append({
        "text": " ".join(texts),
        "box": [
            [x_min, y_min],
            [x_max, y_min],
            [x_max, y_max],
            [x_min, y_max],
        ],
        "prob": min(int(data["conf"][i]) for i in indices) / 100.0
    })

# -----------------------------
# Sort lines top → bottom
# -----------------------------
line_texts.sort(key=lambda x: x["box"][0][1])
for line in line_texts:
    print(line["text"], line["box"], line["prob"])

Cash Basis [[8, 15], [77, 15], [77, 25], [8, 25]] 0.54
Status, [[8, 30], [47, 30], [47, 40], [8, 40]] 0.38


In [2]:
import matplotlib
matplotlib.use("Agg")   # safe for background / GUI apps

import matplotlib.pyplot as plt
import numpy as np

# Data: 20 bars
x = np.arange(1, 21)               # 1 to 20
values = np.random.randint(47, 50, size=20)

# Color mapping
norm = plt.Normalize(values.min(), values.max())
colors = plt.cm.viridis(norm(values))

# Figure
fig, ax = plt.subplots(figsize=(12, 5))

# Bar plot
bars = ax.bar(x, values, color=colors)

# Value labels on top of bars
for bar, value in zip(bars, values):
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height(),
        f"{value}",
        ha="center",
        va="bottom",
        fontsize=9,
        fontweight="bold"
    )

# Colorbar
sm = plt.cm.ScalarMappable(cmap="viridis", norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax)
cbar.set_label("Value Scale")

# Axes labels and title
ax.set_title("Bar Plot with 20 Bars (1–20)")
ax.set_xlabel("Index")
ax.set_ylabel("Value")

ax.set_xticks(x)
ax.set_xticklabels([str(i) for i in x])

fig.tight_layout()
fig.savefig("bar_plot_1_to_20.png", dpi=150)
plt.close(fig)


In [6]:
import matplotlib
matplotlib.use("Agg")  # safe for background / GUI apps

import matplotlib.pyplot as plt

# ------------------------
# Create figure
# ------------------------
fig, ax = plt.subplots(figsize=(8, 5))
ax.axis("off")

# ------------------------
# Title
# ------------------------
ax.text(
    0.5, 0.92,
    "Text Gap Analysis",
    ha="center",
    va="center",
    fontsize=16,
    fontweight="bold"
)

# ========================
# Horizontal gap example
# ========================
ax.text(0.25, 0.65, "Hello", fontsize=14, ha="center")
ax.text(0.75, 0.65, "World!", fontsize=14, ha="center")

# Horizontal arrow
ax.annotate(
    "",
    xy=(0.65, 0.65),
    xytext=(0.35, 0.65),
    arrowprops=dict(arrowstyle="<->", linewidth=2)
)

# Horizontal gap label with padding
ax.annotate(
    "Horizontal Gap = 20",
    xy=(0.5, 0.65),          # arrow center
    xytext=(0, 12),          # padding in points (x, y)
    textcoords="offset points",
    ha="center",
    va="bottom",
    fontsize=12,
    bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray")
)

# ========================
# Vertical gap example
# ========================
ax.text(0.5, 0.45, "Hello", fontsize=14, ha="center")

# Vertical arrow
ax.annotate(
    "",
    xy=(0.5, 0.30),
    xytext=(0.5, 0.40),
    arrowprops=dict(arrowstyle="<->", linewidth=2)
)

# Vertical gap label with padding
ax.annotate(
    "Vertical Gap = 10",
    xy=(0.5, 0.35),          # arrow center
    xytext=(12, 0),          # padding in points (x, y)
    textcoords="offset points",
    ha="left",
    va="center",
    fontsize=12,
    bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray")
)

ax.text(0.5, 0.25, "World", fontsize=14, ha="center")

# ------------------------
# Save image
# ------------------------
fig.savefig(
    "text_gap_analysis.png",
    dpi=150,
    bbox_inches="tight"
)

plt.close(fig)


In [24]:
# write them into json file
import json
with open("output.json", "w") as f:
    json.dump(line_texts, f, indent=4)